# Try to see the issues with spacy in IE & NER


In [ ]:
#Problems of NER illustration through Spacy.
!python -m spacy download en_core_web_lg

import spacy
nlp = spacy.load("en_core_web_lg")
mytext = """SAN FRANCISCO — Shortly after Apple used a new tax law last year to bring back most of the $252 billion it had held abroad, the company said it would buy back $100 billion of its stock.

On Tuesday, Apple announced its plans for another major chunk of the money: It will buy back a further $75 billion in stock.

“Our first priority is always looking after the business and making sure we continue to grow and invest,” Luca Maestri, Apple’s finance chief, said in an interview. “If there is excess cash, then obviously we want to return it to investors.”

Apple’s record buybacks should be welcome news to shareholders, as the stock price is likely to climb. But the buybacks could also expose the company to more criticism that the tax cuts it received have mostly benefited investors and executives.
"""
doc = nlp(mytext)
for ent in doc.ents:
    print(ent.text, "\t", ent.label_)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
SAN FRANCISCO 	 GPE
Apple 	 ORG
last year 	 DATE
$252 billion 	 MONEY
$100 billion 	 MONEY
Tuesday 	 DATE
Apple 	 ORG
$75 billion 	 MONEY
first 	 ORDINAL
Luca Maestri 	 PERSON
Apple 	 ORG
Apple 	 ORG


In [ ]:
count=0 #We see 6 sentences as humans in this text. How many does Spacy see?
for sent in doc.sents:
    print(sent.text)
    print("***End of sent****")
    count = count+1
print("Total sentences: ", count)

SAN FRANCISCO — Shortly after Apple used a new tax law last year to bring back most of the $252 billion it had held abroad, the company said it would buy back $100 billion of its stock.


***End of sent****
On Tuesday, Apple announced its plans for another major chunk of the money: It will buy back a further $75 billion in stock.


***End of sent****
“Our first priority is always looking after the business and making sure we continue to grow and invest,” Luca Maestri, Apple’s finance chief, said in an interview.
***End of sent****
“If there is excess cash, then obviously we want to return it to investors.”


***End of sent****
Apple’s record buybacks should be welcome news to shareholders, as the stock price is likely to climb.
***End of sent****
But the buybacks could also expose the company to more criticism that the tax cuts it received have mostly benefited investors and executives.

***End of sent****
Total sentences:  6


# Train NER model with spacy on CONLL **dataset**

In [ ]:
# Remove newer version (if installed)
!pip uninstall -y datasets

# Install a stable version that supports conll2003
!pip install datasets==2.14.6


Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling mu

In [ ]:
from datasets import load_dataset

# Download CoNLL-2003 NER dataset
dataset = load_dataset("conll2003")
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [ ]:
type(dataset["train"])

datasets.arrow_dataset.Dataset

In [ ]:
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names

def decode_labels(tag_ids):
    return [label_names[i] for i in tag_ids]

example = dataset["train"][0]
tokens = example["tokens"]
ner_labels = decode_labels(example["ner_tags"])

print(list(zip(tokens, ner_labels)))


[('EU', 'B-ORG'), ('rejects', 'O'), ('German', 'B-MISC'), ('call', 'O'), ('to', 'O'), ('boycott', 'O'), ('British', 'B-MISC'), ('lamb', 'O'), ('.', 'O')]


Spacy expects data in this format
--------------------------------
TRAIN_DATA = [
    ("Apple is looking at buying U.K. startup for $1 billion", {
        "entities": [(0, 5, "ORG"), (27, 31, "GPE"), (44, 54, "MONEY")]
    }),
    ("Barack Obama was born in Hawaii.", {
        "entities": [(0, 12, "PERSON"), (25, 31, "GPE")]
    }),
]


In [ ]:
TRAIN_DATA = [
    ("Apple is looking at buying U.K. startup for $1 billion", {
        "entities": [(0, 5, "ORG"), (27, 31, "GPE"), (44, 54, "MONEY")]
    }),
    ("Barack Obama was born in Hawaii.", {
        "entities": [(0, 12, "PERSON"), (25, 31, "GPE")]
    }),
]


In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from __future__ import unicode_literals, print_function
# import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example

In [ ]:
model = None
output_dir=Path("./")
n_iter=100

Convert CONLL data format to spacy format

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("conll2003")
label_names = dataset["train"].features["ner_tags"].feature.names

def conll_to_spacy_format(dataset_split):
    formatted_data = []

    for item in dataset_split:
        tokens = item["tokens"]
        tags = item["ner_tags"]
        entities = []

        # Rebuild full sentence from tokens
        text = " ".join(tokens)
        current = 0

        for token, tag in zip(tokens, tags):
            start = text.find(token, current)
            end = start + len(token)
            label = label_names[tag]

            if label != "O":
                if label.startswith("B-") or not entities or entities[-1][2] != label[2:]:
                    entities.append((start, end, label[2:]))

            current = end

        formatted_data.append((text, {"entities": entities}))

    return formatted_data

# Apply on the training data
TRAIN_DATA = conll_to_spacy_format(dataset["train"])

# Example
# print(spacy_train_data[0])


In [ ]:
#load the model

if model is not None:
    nlp = spacy.load(model)
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')
    print("Created blank 'en' model")

#set up the pipeline

nlp.add_pipe("ner", last=True)


Created blank 'en' model


In [ ]:

# Step 2: Begin training
optimizer = nlp.begin_training()

for i in range(10):  # epochs
    print(f"Epoch {i+1}")
    losses = {}
    for text, annotations in tqdm(TRAIN_DATA):
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, losses=losses)
    print("Losses:", losses)

# Step 3: Save your model
nlp.to_disk("ner_model")
print("Model saved!")


Epoch 1


100%|██████████| 14041/14041 [06:02<00:00, 38.71it/s]


Losses: {'ner': np.float32(11496.662)}
Epoch 2


100%|██████████| 14041/14041 [06:08<00:00, 38.11it/s]


Losses: {'ner': np.float32(6215.3916)}
Epoch 3


...

Losses: {'ner': np.float32(2434.506)}
Epoch 9


100%|██████████| 14041/14041 [06:11<00:00, 37.79it/s]


Losses: {'ner': np.float32(2245.1025)}
Epoch 10


100%|██████████| 14041/14041 [06:16<00:00, 37.30it/s]


Losses: {'ner': np.float32(2188.4038)}
Model saved!


In [ ]:
for _, annotations in TRAIN_DATA:
    print(annotations)
    for ent in annotations.get("entities"):
        print(ent[2])
        ner.add_label(ent[2])

{'entities': [(0, 5, 'ORG'), (27, 31, 'GPE'), (44, 54, 'MONEY')]}
ORG
GPE
MONEY
{'entities': [(0, 12, 'PERSON'), (25, 31, 'GPE')]}
PERSON
GPE
